# Ground motion simulation with Kinematic source model
## Generate adaptively-refined mesh informed by 3D velocity model (Wellington Basin)
#### Created By S. Yuan
#### Modified by D. Li (d.li@gns.cri.nz)
#### 22 Nov. 2023
#### NSHM-SRM

## Step 1. prepare 3D velocity model

In [1]:
# load modules
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
import numpy as np
from netCDF4 import Dataset
from scipy.interpolate import interp1d
import pandas as pd


OSError: /lib64/libm.so.6: version `GLIBC_2.23' not found (required by /scale_wlg_nobackup/filesets/nobackup/gns04005/daisy/PythonDir/lib/python3.10/site-packages/lib/libgmsh.so.4.11)

In [174]:
# Engineering Model elastic properties (uniform velocity)
# E = 1.0e10
# nu = 0.3
# rho_model = 1.9e3

# # first Lame parameter
# lamb = E * nu / (1 + nu) / (1 - 2*nu)

# or using nu and shear modulus
# lamb = G *  2 * nu * (1-2*nu)

# # shear modulus
# G = E / (1 + nu) / 2.

# vp_model = np.sqrt( (lamb + 2* G)/rho_model)
# vs_model = np.sqrt(G/rho_model)

# print(vp_model,vs_model,rho_model)

In [4]:
# load station list, crop data, select attributes required and convert to XYZ coords for SeisSol input
from FigFunc.load_site_csv import load_site_csv

In [5]:
# siteFile = '/Users/duoli/Documents/NSHM_SRM/site_table.csv'
siteFile = '/Users/duoli/Library/Mobile Documents/com~apple~CloudDocs/NSHM/Model_kinematic/site_subtable.csv'
site_lon,site_lat,site_elev = load_site_csv(siteFile)


Index(['Unnamed: 0.1', 'Unnamed: 0', 'net', 'sta', 'lat', 'lon', 'elev',
       'site_class', 'Vs30', 'Vs30_std', 'Q_Vs30', 'Vs30_Ref', 'T0', 'T0_std',
       'Q_T0', 'D_T0', 'T0_ref', 'Z1.0', 'Z1.0_std', 'Q_Z1.0', 'Z1.0_ref',
       'Z2.5', 'Z2.5_std', 'Q_Z2.5', 'Z2.5_ref', 'site_domain_no', 'basin'],
      dtype='object')


## 1.1 Load Basin fine-resolution data/structure
## 1.2 Create 3D basin structure netcdf

In [15]:
# check the Wellington Basin data at 
def load_origin_csv(csvfile,yprofile):
    '''interpolate series (unstructured) data from website onto structured grids'''
    
    table1 = pd.read_csv(csvfile)
    print(table1.keys())

    grdx = table1['X']
    grdy = table1['Y']
    grdz = table1['Z']   
    grdvel = table1['Vs_BLOCK']
    grdrho = table1['DENSITY']


    grd_z = np.unique(grdz)
    grd_x = np.unique(grdx)
    grd_y = np.unique(grdy)
    
    return grd_y, table1[table1.Y==yprofile]

# def load_density_csv(csvfile):
#     '''interpolate series (unstructured) data from website onto structured grids'''
    
#     table1 = pd.read_csv(csvfile)
#     print(table1.keys())

#     grdx = table1['X']
#     grdy = table1['Y']
#     grdz = table1['Z']   
#     grdvel = table1['Vs_BLOCK']

#     grd_z = np.unique(grdz)
#     grd_x = np.unique(grdx)
#     grd_y = np.unique(grdy)
    
#     return grd_y, table1[table1.Y==5429259]

In [56]:
csvfile = '../basin_structure/WGTN3D_v5_VsModel_density.csv'

# Y = 5429259.01

grdy, subtable = load_origin_csv(csvfile,5429259.01)
print(subtable)

# plt.figure()
# plt.scatter(subtable.X,subtable.Z,c=subtable.Vs_BLOCK,cmap='turbo');
# plt.savefig('Y-prof.png',dpi=150)
# plt.show()


Index(['X', 'Y', 'Z', 'WGTN3D_v5', 'GSURFDIST', 'Vs_CALC', 'Vs_BLOCK',
       'DENSITY'],
      dtype='object')
Empty DataFrame
Columns: [X, Y, Z, WGTN3D_v5, GSURFDIST, Vs_CALC, Vs_BLOCK, DENSITY]
Index: []


## 1.2 Load Basin structure data and create Netcdf

In [17]:
# import pre-defined function
from FigFunc.load_vel_csv import load_vel_csv

In [193]:
# from netCDF4 import Dataset    # Note: python is case-sensitive!
csvfile = '../basin_structure/WGTN3D_v5_VsModel_density.csv'

Nx = 40
Nz = 129

grdx,grdy,grdz, grdvel,grdrho = load_vel_csv(csvfile,Nx,Nz)

print(grdvel.shape, grdy.shape,grdz.shape)
print(grdvel.min(),grdrho.min())


Index(['X', 'Y', 'Z', 'WGTN3D_v5', 'GSURFDIST', 'Vs_CALC', 'Vs_BLOCK',
       'DENSITY'],
      dtype='object')
129 215 224
(129, 40, 40) (40,) (129,)
0.0 1.0


In [82]:
## extract several downdip profile for 1D vel

fig,ax = plt.subplots(nrows=1,ncols=2,figsize=(6,4))

ax[0].plot(grdvel[:,5,5],grdz,'-k',label='bedeck')
ax[0].plot(grdvel[:,74,70],grdz,'-r',label='sendiment')
ax[0].plot(grdvel[:,49,49],grdz,'-b',label='reference')
ax[0].legend()

ax[1].plot(grdrho[:,5,5],grdz,'-k',label='bedeck')
ax[1].plot(grdrho[:,74,70],grdz,'-r',label='sendiment')
ax[1].plot(grdrho[:,49,49],grdz,'-b',label='reference')
ax[1].legend()
plt.savefig('1dvel_basinEdge.png')

<IPython.core.display.Javascript object>

In [91]:
## write netcdf file fo Paraview

def write_netcdf(fn,Nx,Ny,Nz,grdvel):
    
    ncfile = Dataset(fn, 'w', format='NETCDF4')
    
    ncfile.createDimension('x', Nx) # latitude axis
    ncfile.createDimension('y', Nx) # longitude axis
    ncfile.createDimension('z', Nz) # longitude axis
    
    ncfile.createVariable('x','f8',('x',))
    ncfile.createVariable('y','f8',('y',))
    ncfile.createVariable('z','f8',('z',))
    ncfile.variables['x'][:] = np.unique(grdx);
    ncfile.variables['y'][:] = np.unique(grdy);
    ncfile.variables['z'][:] = np.unique(grdz)[0:Nz]*5;
    
    vs_data = ncfile.createVariable('Vs','float64', ('z','y','x')) # note: unlimited dimension is leftmost
    vs_data[:,:,:] = grdvel[:,:,:]
    print(vs_data.dtype)
    
    # first print the Dataset object to see what we've got
    print(ncfile)
    # close the Dataset.
    ncfile.close()
    print('Dataset is closed!')

## 1.3 create Netcdf SeisSol input

In [85]:
# if 3d basin structure (rho, mu, lambda)

def write_netcdf_inp(fname,Nx, Nz, grdvel, grdrho):
    
    # vp_orig, z_orig, rho_orig is from national wide 1d model
    # vp_1d = np.interp(grdz,z1d,vp1d)
    # rho_1d = np.interp(grdz,z1d,vp1d)

    nu = 0.25
    
    # print(lam_1d,rho_1d)
    
# or using nu and shear modulus
# lamb = G *  2 * nu * (1-2*nu)

# # shear modulus
# G = E / (1 + nu) / 2.

# vp_model = np.sqrt( (lamb + 2* G)/rho_model)
# vs_model = np.sqrt(G/rho_model)
    
    # in case uniform 
    vp = 2600.32
    rho = 1900.0

    ### output for ASAGI format
    fout = Dataset(fname,'w',format='NETCDF4')
    # fview = Dataset('matBasin_mu_3d_view.nc','w+',format='NETCDF4')

    fout.createDimension('x',Nx)
    fout.createDimension('y',Nx)
    fout.createDimension('z',Nz)
    fout.createVariable('x','f8',('x',))
    fout.createVariable('y','f8',('y',))
    fout.createVariable('z','f8',('z',))
    fout.variables['x'][:] = np.unique(grdx);
    fout.variables['y'][:] = np.unique(grdy);
    fout.variables['z'][:] = np.unique(grdz);
    
    material = np.dtype([('rho',np.float32),('mu',np.float32),('lambda',np.float32)])
    data = fout.createCompoundType(material,'material')
    vv = fout.createVariable('data',data,dimensions=('z','y','x'))
    # vv = fout.createVariable('mu','float64', ('z','y','x')) # note: unlimited dimension is leftmost

    for i in np.arange(Nx):
        for j in np.arange(Nx):
            rho_1d = grdrho[:,j,i]
            mu_1d = grdvel[:,j,i] **2 * rho_1d # convert to 1d mu profile
            lam_1d = 2 * mu_1d * nu * (1 - 2*nu) 
            # print(mu_1d.shape, lam_1d.shape)
            for k in range(Nz):
                vv[k,j,i] = (rho_1d[k], mu_1d[k],lam_1d[k])# 3D velocity model
    
    fout.close()

In [115]:
# create netcdf file
# fn = 'WelVel_shallow_extra.nc'

# write a Paraview file
# write_netcdf(fout,Nx,Ny,Nz,grdvel)


In [86]:
# print(grdx.min(),grdx.max(),grdy.min(),grdy.max())
# print(grdz, z_orig[::-1],vp_orig[::-1])

fout = 'matBasin_rho_mu_lam_basin.nc'
# print(grdz)  # every 10 m in depth

write_netcdf_inp(fout,Nx,Nz,grdvel,grdrho)

## 1.4 Create Synthetic receiver locations around the basin edge

In [86]:
## load one profile extracted from Paraview
## synthetic stations as basin base and surface 

csvProf = 'Receivers/ProfileData.csv'
tabProf = pd.read_csv(csvProf)
synsite = np.array([np.array(tabProf['Points:0']),np.array(tabProf['Points:1']),np.array(tabProf['Points:2'])])
# print(synsite.shape, synsite)


sub_tabProf = tabProf[tabProf['vs']<200]
print(sub_tabProf.shape)

synsite = np.array([np.array(sub_tabProf['Points:0']),np.array(sub_tabProf['Points:1']),np.array(sub_tabProf['Points:2'])])
np.savetxt('Receivers/synsite_surf_xyz.txt',synsite.transpose())


# plt.figure()
# plt.scatter(tabProf['Points:0'],tabProf['Points:2'],c=tabProf['vs'],cmap='turbo')
# plt.plot(sub_tabProf['Points:0'],sub_tabProf['Points:2'],'*w')

# plt.savefig('Receivers/synProf1.png')
# plt.show()

(177, 6)


In [145]:
# load data Vertical scale up by 10
# BasinEdge and BasinTopo receivers

csvProf = 'Receivers/vsGradslide.csv'
gradProf0 = pd.read_csv(csvProf)
print(gradProf0.shape)

xdot = np.unique(np.array(gradProf['Points:0']))
ydot = np.unique(np.array(gradProf['Points:1']))
zdot = np.unique(np.array(gradProf['Points:2']))

print(ydot.size*zdot.size)

gradProf = gradProf0[gradProf0['vs']<1000]

ydot = np.unique(np.array(gradProf['Points:1']))
print(ydot.size)

siteBasinEdge = np.zeros((ydot.size,3))
siteBasinTopo = np.zeros((ydot.size,3))
print(siteBasinEdge.shape)

for i in range(ydot.size):
    sele = gradProf[gradProf['Points:1']==ydot[i]]
    # print(sele['Points:2'].values)
    siteBasinEdge[i,:] = np.array([sele['Points:0'].values[0],sele['Points:1'].values[0],1/10*sele['Points:2'].values[0]-50.0])
    siteBasinTopo[i,:] = np.array([sele['Points:0'].values[0],sele['Points:1'].values[0],1/10*sele['Points:2'].values[-1]])

# siteBasin = np.array([np.array(gradProf['Points:0']),np.array(gradProf['Points:1']),np.array(gradProf['Points:2'])])

# print(siteBasinEdge)

# np.savetxt('Receivers/synBasinEdge.txt',siteBasinEdge)
# np.savetxt('Receivers/synBasinTopo.txt',siteBasinTopo)


plt.figure(figsize=(7,4))
plt.scatter(gradProf0['Points:0'],gradProf0['Points:2']/10 ,c=gradProf0['vs'],cmap='turbo',alpha=0.5)
plt.plot(siteBasinEdge[:,0],siteBasinEdge[:,2],'^k')
plt.plot(siteBasinTopo[:,0],siteBasinTopo[:,2],'^',markeredgecolor='k',markerfacecolor='w')

plt.colorbar()
plt.savefig('Receivers/synG3.png')
plt.show()

(11340, 9)
2666
43
(43, 3)


<IPython.core.display.Javascript object>

In [150]:
## Load station information and select station accordingly

from evtk.hl import pointsToVTK 


ModuleNotFoundError: No module named 'evtk'

In [232]:
# check and Select stations in area Wellington and crop data

siteFile = '/Users/duoli/Documents/NSHM_SRM/site_table.csv'

table1 = pd.read_csv(siteFile)

exd = 0.15
min_lat = -41.4-exd
max_lat = -41.197+exd
max_lon = 174.903 +exd
min_lon = 174.7-exd

table1_select = table1[(table1['lon']>min_lon )& (table1['lon']<max_lon) & (table1['lat']>min_lat )& (table1['lat']<max_lat)]

site_lon = table1_select['lon']
site_lat = table1_select['lat']
site_elev = table1_select['elev']
site_name = table1_select['sta']

siteTable = pd.DataFrame(table1_select)
siteTable.to_csv('site_subtable2.csv')
print(table1_select.shape, print(site_name))

17     ARKS
32      BHW
36     BMTS
38     BOWS
73     CUBS
83     DAVS
105    EBPS
122    FAIS
127    FKPS
147    GPSS
168    HIBS
180    HSSS
190    INSS
243    LHBS
244    LHES
245    LHRS
246    LHUS
248    LIRS
259    LNBS
276    LRSS
311    MISS
313    MKBS
315    MKVS
334     MRW
358    NBSS
363    NEWS
408     OTW
418    PFAS
420    PGMS
421    PHFS
422    PHHS
423    PIPS
438    POKS
439    POLS
440    POTS
446    PRKS
452    PTOS
455    PVCS
456    PWES
493    RQGS
510    SEAS
513    SEVS
525    SNZO
526    SOCS
527    SOMS
540    TAIS
553    TEPS
555    TFSS
586    TRTS
604    UHCS
610    VUWS
617    WANS
624    WCFS
627    WDAS
630     WEL
631    WEMS
648    WNAS
649    WNHS
650    WNKS
665    WTYS
Name: sta, dtype: object
(60, 25) None


In [235]:
# crop the data based on the range of long and lat

siteFile = '/Users/duoli/Documents/NSHM_SRM/site_subtable2.csv'
subtable = pd.read_csv(siteFile)

data = np.array([subtable.lon[:],subtable.lat[:],subtable.elev[:]])
print(data.shape)

# convert  # UTM projection
lla = pyproj.Proj(proj='latlong', ellps='WGS84', datum='WGS84')
myproj = pyproj.Proj(init='epsg:2193', datum='WGS84')

site_map = pyproj.transform(lla, myproj, subtable.lon,subtable.lat,subtable.elev, radians=False)
x_site = site_map[0]
y_site = site_map[1]

site_xyz = np.array([x_site[:],y_site[:],site_elev[:]])
print(data.shape)

np.savetxt('stations2_xyz.txt', site_xyz.transpose())
np.savetxt('stations2_lonlat.txt', data.transpose())


(3, 60)
(3, 60)


/Users/duoli/anaconda3/envs/py39/lib/python3.9/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/var/folders/w1/_6wbwxqn58j2p6t9v4n73wcr0000gn/T/ipykernel_57837/2028014924.py:13: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  site_map = pyproj.transform(lla, myproj, subtable.lon,subtable.lat,subtable.elev, radians=False)


## 1.2. 1D velocity profile from National wide model

In [343]:
# reference https://zenodo.org/records/3779523
# nzvel 2.2
# notes: use the most recent near-coast result?

xgrids = np.array([-1200., -540., -400., -300., -200., -151., -136., -121., -106. , -96. , -86. , -76. , -68.,  -61. , -53.,  -46.,  -38. ,
                   -31. , -23.,  -16., -8. ,  -1.  ,  7.,   14. ,  22. ,  29. ,  37. ,  41.,   44. ,  48. ,  51. ,  55.,   58. ,  62. ,  65. ,
                   69. ,  72.,   76. ,  79.,   83., 86. ,  90.  , 93. ,  97. , 100.,  104.,  107. , 111. , 114.,  118. , 121. , 127. ,
                   133. , 139.,  145.,  152. , 161. , 178. , 205. , 230., 260.,  300. , 350., 1200.,])
ygrids= np.array([-1200., -703., -653. ,-632., -610., -590., -573., -554., -536. ,-515., -494., -468., -439., -409.,
                  -379. ,-349., -329., -310., -291. ,-272., -253., -228., -198., -173., -148., -123., -105. , -90.,  
                  -80. , -70. , -60.,  -50. , -40. , -30. , -20.,  -10.,    0. ,  10.,   20. ,  30.,
                  40.,   55. ,  70.  , 85. , 100. , 115. , 130. , 145. , 155.  ,163. , 171.  ,179. , 187. , 197. , 224. , 269.,  313. , 358.  ,388.,  418.,
                  448. , 477. , 507. , 537.,  567.,  600.,  639.,  686.,  736., 1200.])
zgrids = np.array([ -15. ,  -1.  , 1.  ,  3.,    5.,    8. ,  15.  , 23. ,  30. ,  34.,   38.,   42.,   48. ,  55.,  65. ,  85.,  105.,
                   130. , 155. , 185.,225. , 275.,  370. , 620.,  750.])

print(xgrids.size,ygrids.size,zgrids.size)


## The models use Transverse Mercator coordinate transformation with a central meridian= 173,
# and counterclockwise rotation of 140.  

velproj = pyproj.Proj(proj='tmerc',lon_0=173, lat_0=-42, datum='WGS84')
lla = pyproj.Proj(proj='latlong', ellps='WGS84', datum='WGS84')

# project it back to long and lat
qx,qy = rotate((x_matx[5,:,:],y_matx[5,:,:]),(0,0),140)
# project XYZ coordinates to Longlat

# this will be used as the new model
vel_lon, vel_lat = pyproj.transform(velproj,lla, qx*1e3,qy*1e3, radians=False)

print(vel_lon.shape, vel_lat)

64 70 25
(70, 64) [[-40.10169723 -47.52962934 -49.11076185 ... -56.84183081 -57.37243673
  -65.37946178]
 [-36.56312209 -43.60095802 -45.07505703 ... -52.12112696 -52.592986
  -59.46900113]
 [-36.17070505 -43.16769225 -44.6309862  ... -51.6112575  -52.07772762
  -58.85775543]
 ...
 [-23.80226499 -29.8237676  -31.04603877 ... -36.68491234 -37.05037986
  -42.24704716]
 [-23.28810983 -29.28151567 -30.49728261 ... -36.10316846 -36.46638368
  -41.63338476]
 [-18.40848997 -24.17851419 -25.34344369 ... -30.69853146 -31.04518115
  -36.00992342]]


/var/folders/w1/_6wbwxqn58j2p6t9v4n73wcr0000gn/T/ipykernel_57837/2521587706.py:31: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  vel_lon, vel_lat = pyproj.transform(velproj,lla, qx*1e3,qy*1e3, radians=False)


In [344]:
# Find Wellington city

# selected velocity range
plt.figure()
plt.scatter(vel_lon[25:50,15:55],vel_lat[25:50,15:55],s = 30, c = vs_matx[3,25:50,15:55])

plt.plot(coast[0,:],coast[1,:],'-k',linewidth=1.2)

plt.xlim(172,177) 
plt.ylim(-45,-35)

plt.savefig('3Dvel_NZd3.png',dpi=300)
plt.show()



<IPython.core.display.Javascript object>

In [240]:
import math

def rotate(origin, point, angle):
    """
    Rotate a point counterclockwise by a given angle around a given origin.

    The angle should be given in radians.
    """
    ox, oy = origin
    px, py = point

    qx = ox + math.cos(angle) * (px - ox) - math.sin(angle) * (py - oy)
    qy = oy + math.sin(angle) * (px - ox) + math.cos(angle) * (py - oy)
    return qx, qy

In [ ]:
## 

In [347]:
# laod velocity model from https://zenodo.org/records/3779523

vfile = open('../Velocity/vlnzw2p2dnxyzltln.tbl.txt','r')
vdata = np.loadtxt(vfile,skiprows=2)
print(vdata.shape)

# reshape into matrix with index of [z,y,x]
vp_matx = np.reshape(vdata[:,0],[25,70,64])
vs_matx = np.reshape(vdata[:,2],[25,70,64])
rho_matx = np.reshape(vdata[:,3],[25,70,64])

# Coordinates
# x_matx = np.reshape(vdata[:,6],[25,70,64])
# y_matx = np.reshape(vdata[:,7],[25,70,64])

# Find Wellington city
nxd = np.abs(xgrids - 100).argmin()
nyd = np.abs(ygrids + 150).argmin()

print(nxd,nyd)

##
# nxd,nyd = 43,25
print(xgrids[nxd],ygrids[nyd])
print(vs_matx[:,25,43])
print(vs_matx[:,nyd,nxd])


(112000, 11)
44 24
100.0 -148.0
[1.93 2.51 2.76 2.84 3.04 3.4  3.74 3.92 3.99 4.42 4.81 5.07 5.1  5.19
 5.05 4.81 4.8  4.82 4.79 4.84 4.92 4.98 5.16 5.9  6.13]
[2.01 2.65 2.9  3.42 3.68 3.73 3.5  3.81 4.17 4.49 4.83 4.99 5.06 5.19
 5.26 4.84 4.72 4.74 4.77 4.81 4.89 4.95 5.18 5.9  6.13]


In [104]:
fig,ax = plt.subplots(1,1,figsize=(5,5))
# ax.imshow(vp_matx[1,:,:],interpolation='bilinear', cmap='plasma_r',aspect='auto', vmax=5, vmin=1)

ax.plot(vp_matx[:,nyd,nxd],zgrids,'-b',label='Vp')
# ax.plot(vp,z,'--b',label='Vp')

ax.plot(vs_matx[:,nyd,nxd],zgrids,'-r',label='Vs')
ax.plot(rho_matx[:,nyd,nxd],zgrids,'-k',label='rho')
plt.legend()
plt.gca().invert_yaxis()
plt.savefig('1Dvel_nz.png',dpi=100)

<IPython.core.display.Javascript object>

In [353]:
# velocity model is from XXX
# conver to SI unit

# vp_orig = vp_matx[:,nyd,nxd]*1000
# vs_orig = vs_matx[:,nyd,nxd]*1000
# rho_orig = rho_matx[:,nyd,nxd]*1000

# select 3D
# xgrid_3d = xgrid[15:55]
# ygrid_3d = ygrid[25:50]

vs_3d = vs_matx[:,25:50,15:55]*1000
vp_3d = vs_matx[:,25:50,15:55]*1000
rho_3d = vs_matx[:,25:50,15:55]*1000

# Coordinates
x_matx = np.reshape(vdata[:,6],[25,70,64])
y_matx = np.reshape(vdata[:,7],[25,70,64])

velproj = pyproj.Proj(proj='tmerc',lon_0=173, lat_0=-42, datum='WGS84')
lla = pyproj.Proj(proj='latlong', ellps='WGS84', datum='WGS84')

# project it back to long and lat
qx,qy = rotate((x_matx[5,25:50,15:55],y_matx[5,25:50,15:55]),(0,0),140)

x1,y1 = pyproj.transform(lla,myproj,173,-42, radians=False)

print(x1,y1)

# origin = (1746511.3065337986, 5426462.244420674)


# reverse z axis to be consistent with SeisSol
z_orig = -zgrids*1e3

# resample Z steps: uniformly sampled
z = np.linspace(-20e3,400,1000)

# interpolate to the uniformly-sampled grids

# vp_uni = np.interp(z,z_orig[::-1],vp_orig[::-1])
# vs_uni = np.interp(z,z_orig[::-1],vs_orig[::-1])
# rho_uni = np.interp(z,z_orig[::-1],rho_orig[::-1])

# print(vs)
# print(z)
# Depth of the Earth model 
# zmin= 0.
# zmax=-5515.

# z=np.linspace(zmin,zmax,1104) 

# Add Vs30 and Vs100
# for num_z in np.arange(0,20):
#     if (z[num_z] >= -30.):
#         vp[num_z] = 900.
#         vs[num_z] = 500.
#     elif (z[num_z] < -30. and z[num_z] >= -100.):
#         vp[num_z] = 1500.
#         vs[num_z] = 900.        

# Set elastic properties for buildings
# z_building = np.arange(6.5,0.0,-0.05)
# z  = np.linspace(zmin,zmax,1104)
# z  = np.insert(z,  0, z_building)
# vp = np.insert(vp, 0, np.ones(len(z_building))*vp_model)
# vs = np.insert(vs, 0, np.ones(len(z_building))*vs_model)
# rho= np.insert(rho,0, np.ones(len(z_building))*rho_model)
# print(vs)

1600000.0 5350223.775180822


/var/folders/w1/_6wbwxqn58j2p6t9v4n73wcr0000gn/T/ipykernel_57837/2082545432.py:26: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  x1,y1 = pyproj.transform(lla,myproj,173,-42, radians=False)


In [ ]:
# Write 3D velocity model for Wellington Basin

# if 1D used
# rho_d  = rho_uni
# mu_d   = rho_d * vs_uni **2
# lamb_d = rho_d * vp_uni**2 - 2.0 * mu_d

# nx_d = 5
# ny_d = 5
# nz_d = np.shape(rho_d)[0]

# # extend the range wide to save the interpolate

# x_d=np.linspace(xmin-10e3,xmax+10e3,nx_d) 
# y_d=np.linspace(ymin-10e3,ymax+10e3,ny_d)
# z_d=z 
# z_d[-1] = 10e3


# ### output for ASAGI format
# fout = Dataset('matBasin_rho_mu_lam_3d.nc','w',format='NETCDF4')
# # fout = Dataset('matBasin_rho_mu_lam_1d.nc','w',format='NETCDF4')

# fout.createDimension('x',nx_d)
# fout.createDimension('y',ny_d)
# fout.createDimension('z',nz_d)
# fout.createVariable('x','f4',('x',))
# fout.createVariable('y','f4',('y',))
# fout.createVariable('z','f4',('z',))
# fout.variables['x'][:] = x_d;
# fout.variables['y'][:] = y_d;
# fout.variables['z'][:] = z_d;

# # material = np.dtype([('rho',np.float32),('lambda',np.float32)])
# material = np.dtype([('rho',np.float32),('mu',np.float32),('lambda',np.float32)])
# data = fout.createCompoundType(material,'material')
# vv = fout.createVariable('data',data,dimensions=('z','y','x'))

# for j in range(ny_d):
#     for i in range(nx_d):
#         for k in range(nz_d):
#             vv[k,j,i] = (rho_d[k],mu_d[k], lamb_d[k]) # 3D velocity model
#         # vv[:,j,i] = (mu_d,lamb_d) # 3D velocity model


# fout.close()

In [345]:
# if 3D used

### output for ASAGI format
fout = Dataset('matBasin_rho_mu_lam_3d.nc','w',format='NETCDF4')
# fout = Dataset('matBasin_rho_mu_lam_1d.nc','w',format='NETCDF4')
vv = fout.createVariable('data',data,dimensions=('z','y','x'))

nx_d = 25
ny_d = 25
nz_d = np.shape(z_orig)

# extend the range wide to save the interpolate

x_d=np.linspace(xmin-10e3,xmax+10e3,nx_d) 
y_d=np.linspace(ymin-10e3,ymax+10e3,ny_d)
z_d=z 
z_d[-1] = 10e3

fout.createDimension('x',nx_d)
fout.createDimension('y',ny_d)
fout.createDimension('z',nz_d)
fout.createVariable('x','f4',('x',))
fout.createVariable('y','f4',('y',))
fout.createVariable('z','f4',('z',))
fout.variables['x'][:] = x_d;
fout.variables['y'][:] = y_d;
fout.variables['z'][:] = z_d;

# material = np.dtype([('rho',np.float32),('lambda',np.float32)])
material = np.dtype([('rho',np.float32),('mu',np.float32),('lambda',np.float32)])
data = fout.createCompoundType(material,'material')


for j in range(ny_d):
    for i in range(nx_d):
                
        rho_d  = rho_matx[:,j,i]
        mu_d   = rho_matx[:,j,i] * vs_matx[:,j,i] **2
        lamb_d = rho_matx[:,j,i] * vp_matx[:,j,i]**2 - 2.0 * mu_d
        
        for k in range(nz_d):
            vv[k,j,i] = (rho_d[k],mu_d[k], lamb_d[k]) # 3D velocity model
        # vv[:,j,i] = (mu_d,lamb_d) # 3D velocity model


fout.close()

NameError: name 'rho_uni' is not defined

In [ ]:
# fig,ax=plt.subplots(1,1,figsize=(5,5))
# ax.plot(vs,z,'-k',label='Vs');
# ax.plot(vp,z,'-r',label='Vp');
# plt.legend()
# plt.savefig('1D_vel.png',dpi=150,transparent=False)




## Step 2. create mesh file with topography

In [210]:
# This is an example to load Topographic date from Etopo
# load netcdf reader.

from netCDF4 import Dataset
%matplotlib notebook
import pyproj


def load_topo_data(ncfile):
    nc = Dataset(ncfile,'r')
    lon= nc.variables['lon'][:]
    lat = nc.variables['lat'][:]
    topo = nc.variables['z'][:,:]
    lons,lats=np.meshgrid(lon,lat)

    samples = len(lon)
    
    min_latitude= lat.min()
    min_longitude= lon.min()
    
    max_latitude= lat.max()
    max_longitude= lon.max()
    
    mark_x = np.where( topo == -32768 )[0]
    mark_y = np.where( topo == -32768 )[1]
    # for x, y in zip(mark_x, mark_y) :
    #     slice = topo[max(0, x-1):x+1, max(0,y-1):y+1] # assuming a 5x5 square
    #     topo[x,y] = np.mean([i for i in slice.flatten() if i > 0])  # threshold is 0
    
    x_lon = np.linspace((min_longitude),(max_longitude),samples)
    y_lat = np.linspace((min_latitude),(max_latitude),samples)
    
    # UTM projection
    lla = pyproj.Proj(proj='latlong', ellps='WGS84', datum='WGS84')
    # myproj = pyproj.Proj(proj='tmerc',lon_0=168, datum='WGS84')
    # myproj = pyproj.Proj(proj='utm',zone='59', datum='WGS84') # nzmg  
    myproj = pyproj.Proj(init='epsg:2193', datum='WGS84')
    
    xyz_map = pyproj.transform(lla, myproj, x_lon,y_lat,np.zeros(len(x_lon)), radians=False)
    x = xyz_map[0]
    y = xyz_map[1]
    # print(x,y)
    
    # Wellington city Epicenter coordinate
    lat_sou = -41.3
    lon_sou = 174.75 
    xyz_sou = pyproj.transform(lla, myproj, lon_sou,lat_sou, radians=False) # Epicenter in UTM domain
    print(xyz_sou)
    
    xmin = x[0] # Unit (m)
    xmax = x[-1]
    
    ymin = y[0]
    ymax = y[-1]

    return topo, x, y

In [211]:
# These data contain occassional voids from a number of causes such as shadowing, 
# phase unwrapping anomalies, or other radar-specific causes. Voids are flagged with the value -32768.
# I replace them by the average of the surrounding values


In [ ]:
# print(xyz_sou)
# print(x[1]-x[0],y[1]-y[0])
# print(topo.shape)

## Plot topographsic area

In [131]:
# Plot topo data and check if the data is well-resolved

from matplotlib.colors import LightSource

ls = LightSource(azdeg=35, altdeg=45)
cmap = plt.cm.terrain


fig, ax = plt.subplots(1,1,figsize=(7,5))

# Lat/Lon 
# im1 = ax.imshow(topo, interpolation='bilinear', cmap='terrain',aspect='auto',
#                    origin='lower', extent=[x_lon[0], x_lon[-1], y_lat[0], y_lat[-1]+1],
#                    vmax=700, vmin=-1000)
# ax[0].set_title('Lon/Lat',fontsize=14)
# ax[0].set_xlabel('Longitude ($^{\circ}$)', fontsize=12)
# ax[0].set_ylabel('Latitude ($^{\circ}$)', fontsize=12)
# # ax[0].scatter(lon_sou,lat_sou, s=100, marker='.', c='r',label='WEL')
# ax[0].legend(loc=1,prop={"size":12})2

# UTM domain
im2 = ax.imshow(topo, interpolation='bilinear', cmap='terrain',aspect='auto',
                   origin='lower', extent=[xrange[0], xrange[-1], yrange[0], yrange[-1]+1],
                    vmax=700, vmin=-700)

# blend_mode = ['hsv','overlay', 'soft']

# rgb = ls.shade(topo, cmap=cmap, blend_mode='overlay', vert_exag=2, dx=23, dy=30 )
# im2 = ax.imshow(rgb, cmap=cmap,origin='lower',aspect='auto',extent=[x[0], x[-1], y[0], y[-1]+1],vmax=600, vmin=-1000)

ax.scatter(x_site,y_site, s=40, marker='^', c='b')

for k in range(subtable.sta.size):
    ax.text(x_site[k]-100.0,y_site[k]+100.0,s=subtable.sta[k],fontsize=5)

ax.xaxis.major.formatter.set_powerlimits((-2,1))
ax.yaxis.major.formatter.set_powerlimits((-2,1))

ax.set_xlabel('X (m)', fontsize=12)
ax.set_ylabel('Y (m)', fontsize=12)
ax.set_title('NZTM2000 projection',fontsize=14)
ax.legend(loc=1,prop={"size":12})
ax.set_aspect(1)
ax.set_xticklabels([])
ax.set_yticklabels([])

fig.colorbar(im2,ax=ax,label='Elevation (m)', fraction=0.046, pad=0.025, extend='both')

outname = 'TopoWel'+'-basin3.png'
plt.savefig(outname, dpi=150, transparent=False)
plt.show()

<IPython.core.display.Javascript object>

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [130]:
print(subtable.sta,subtable.Vs30)

0     BOWS
1     CUBS
2     EBPS
3     FKPS
4     LHRS
5     LHUS
6     MISS
7     MKVS
8     NEWS
9     PGMS
10    PIPS
11    POTS
12    PTOS
13    PVCS
14    RQGS
15    SEAS
16    SEVS
17    SNZO
18    SOMS
19    TEPS
20    TFSS
21    TRTS
22    VUWS
23    WCFS
24     WEL
25    WEMS
26    WNAS
27    WNHS
28    WNKS
29    WTYS
Name: sta, dtype: object 0      267
1      278
2      540
3      323
4      622
5      212
6      274
7      480
8     1000
9      200
10     210
11     453
12     450
13     190
14     246
15     305
16     209
17     622
18    1000
19     292
20     271
21     270
22     286
23     349
24     687
25     265
26     229
27     493
28     369
29     230
Name: Vs30, dtype: int64


## Create mesh using GmSH

In [212]:
# load topographic data
ncfile = '../Geometry/NZ_gebco_03sB.grd'
# ncfile = '../Geometry/NZ_gebco_01s.grd'

topo, xrange,yrange = load_topo_data(ncfile)

print(topo.shape,xrange.shape)

(1746511.3065337986, 5426462.244420674)
(721, 721) (721,)


/Users/duoli/anaconda3/envs/py39/lib/python3.9/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/var/folders/w1/_6wbwxqn58j2p6t9v4n73wcr0000gn/T/ipykernel_57837/2748703864.py:39: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  xyz_map = pyproj.transform(lla, myproj, x_lon,y_lat,np.zeros(len(x_lon)), radians=False)
/var/folders/w1/_6wbwxqn58j2p6t9v4n73wcr0000gn/T/ipykernel_57837/2748703864.py:47: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  xyz_sou = pyproj.transform(lla, myproj, lon_sou,lat_sou, 

In [213]:
print(xrange.min(),xrange.max(),yrange.min(),yrange.max())

1725003.0943659162 1776618.797887972 5393547.258913646 5459118.953945233


In [214]:
# Initialize Gmsh API

# to add topographic data in Model geometry

xmin = xrange[0] # Unit (m)
xmax = xrange[-1]

ymin = yrange[0]
ymax = yrange[-1]
# print(xmin,xmax,ymin,ymax)

zmax = -15000.
# print(x_lon.size, y_lat.size)


# Create the terrain mesh with N by N data points 
# (Make sure the spatial resolution matching topographic data):
N = 600

# Helper function to return a node tag given two indices i and j:
def tag(i, j):
    return (N + 1) * i + j + 1

# The x, y, z coordinates of all the nodes:
coords = []

# The tags of the corresponding nodes:
nodes = []

# The connectivities of the triangle elements (3 node tags per triangle) on the
# terrain surface:
tris = []

# The connectivities of the line elements on the 4 boundaries (2 node tags
# for each line element):
lin = [[], [], [], []]

# The connectivities of the point elements on the 4 corners (1 node tag for each
# point element):
pnt = [tag(0, 0), tag(N, 0), tag(N, N), tag(0, N)]

# Adding topography point by point
x_grid = np.linspace(xmin,xmax,N+1)
y_grid = np.linspace(ymin,ymax,N+1)
print(x_grid.size,y_grid.size)
print(xmin,xmax,ymin,ymax)

601 601
1725003.0943659162 1776618.797887972 5393547.258913646 5459118.953945233


In [215]:
# Velocity-aware mesh
# try 1d velocity model 

# just an example for test
# vs_mesh = np.linspace(2885,4500,30)
# z_mesh = np.linspace(-20e3,1400,30)

# Wellington basin 1D: extreace with using 
#  grdx,grdy,grdz, grdvel = load_vel_csv(csvfile,Nx,Nz)
vs_mesh = grdvel[:,:,:]
z_mesh =  grdz
x_mesh = grdx
y_mesh = grdy

# check minimum velocity , > 0.0
# print(vs_mesh.min(),np.where(vs_mesh<160))
vs_mesh[np.where(vs_mesh<160)] = 160.0

# print( y_mesh, x_mesh)

lc = 10e3
fre_high = 2.0
num_elements_per_wavelength = 1.5 

# define adaptive mesh refinement scheme
# only used 1D velocity model for adaptive mesh

def cal_mesh_lc(x0,y0,z0):
    lc = 5e3
    if z0 > -1500.0:
        if z0 > 200:
            lc =  500.0 # Densify the mesh for buildings
        else:
            indz = (np.abs(z0-z_mesh)).argmin()
            indy = (np.abs(y0-y_mesh)).argmin()
            indx = (np.abs(x0-x_mesh)).argmin()
            if (np.abs(x0-x_mesh).min()> 800.0) | (np.abs(y0-y_mesh).min()> 800.0 ) |(np.abs(z0-z_mesh).min()> 800.0 ) :
                # print(np.abs(y0-y_mesh).min(), np.abs(x0-x_mesh).min())
                lc = 500.0
            else:
                lc = vs_mesh[indz,indy,indx] / fre_high / num_elements_per_wavelength
                # print(lc)
        return lc
    else:
        return lc

def cal_mesh_size(entity_dim, entity_tag, x, y, z, lc):
    return cal_mesh_lc(x,y,z) 

In [216]:
# check realiablity
cal_mesh_lc(1.75e6, 5.43e6,-200.0)
# print(topo.shape)

233.33333333333334

In [217]:
# create adaptive mesh for basin using 1D velocity profile 

gmsh.initialize()
gmsh.model.add("WelBasin")


for i in range(N + 1):
    for j in range(N + 1):
        nodes.append(tag(i, j))
        ind_x = (np.abs(x_grid[i]-xrange)).argmin() # find the X index of nearest point in topo matrix
        ind_y = (np.abs(y_grid[j]-yrange)).argmin() # find the Y index of nearest point in topo matrix
        coords.extend([x_grid[i],y_grid[j],np.float64(topo[ind_y,ind_x])]) # Add the elevation 
        if i > 0 and j > 0:
            tris.extend([tag(i - 1, j - 1), tag(i, j - 1), tag(i - 1, j)]) 
            tris.extend([tag(i, j - 1), tag(i, j), tag(i - 1, j)])
        if (i == 0 or i == N) and j > 0:
            lin[3 if i == 0 else 1].extend([tag(i, j - 1), tag(i, j)])
        if (j == 0 or j == N) and i > 0:
            lin[0 if j == 0 else 2].extend([tag(i - 1, j), tag(i, j)])

# Create 4 discrete points for the 4 corners of the terrain surface:
for i in range(4):
    gmsh.model.addDiscreteEntity(0, i + 1)
gmsh.model.setCoordinates(1, xmin, ymin, coords[3 * tag(0, 0) - 1])
gmsh.model.setCoordinates(2, xmax, ymin, coords[3 * tag(N, 0) - 1])
gmsh.model.setCoordinates(3, xmax, ymax, coords[3 * tag(N, N) - 1])
gmsh.model.setCoordinates(4, xmin, ymax, coords[3 * tag(0, N) - 1])

# Create 4 discrete bounding curves, with their boundary points:
for i in range(4):
    gmsh.model.addDiscreteEntity(1, i + 1, [i + 1, i + 2 if i < 3 else 1])

# Create one discrete surface, with its bounding curves:
gmsh.model.addDiscreteEntity(2, 1, [1, 2, -3, -4])

# Add all the nodes on the surface:
gmsh.model.mesh.addNodes(2, 1, nodes, coords)
gmsh.model.addPhysicalGroup(2, [1], 101) # Free-surface boundary label

# Add point elements on the 4 points, line elements on the 4 curves, and triangle elements on the surface:
for i in range(4):
    # Type 15 for point elements:
    gmsh.model.mesh.addElementsByType(i + 1, 15, [], [pnt[i]])
    # Type 1 for 2-node line elements:
    gmsh.model.mesh.addElementsByType(i + 1, 1, [], lin[i])
# Type 2 for 3-node triangle elements:
gmsh.model.mesh.addElementsByType(1, 2, [], tris)

# Reclassify the nodes on the curves and the points 
gmsh.model.mesh.reclassifyNodes()

# Create a geometry for the discrete curves and surfaces, so that we can remesh them later on:
gmsh.model.mesh.createGeometry()

# Create other entities to form one volume below the terrain surface:
p1 = gmsh.model.geo.addPoint(xmin, ymin, zmax)
p2 = gmsh.model.geo.addPoint(xmax, ymin, zmax)
p3 = gmsh.model.geo.addPoint(xmax, ymax, zmax)
p4 = gmsh.model.geo.addPoint(xmin, ymax, zmax)
c1 = gmsh.model.geo.addLine(p1, p2)
c2 = gmsh.model.geo.addLine(p2, p3)
c3 = gmsh.model.geo.addLine(p3, p4)
c4 = gmsh.model.geo.addLine(p4, p1)
c10 = gmsh.model.geo.addLine(p1, 1)
c11 = gmsh.model.geo.addLine(p2, 2)
c12 = gmsh.model.geo.addLine(p3, 3)
c13 = gmsh.model.geo.addLine(p4, 4)
ll1 = gmsh.model.geo.addCurveLoop([c1, c2, c3, c4]) 
s1 = gmsh.model.geo.addPlaneSurface([ll1]) # bot
ll3 = gmsh.model.geo.addCurveLoop([c1, c11, -1, -c10]) # fro
s3 = gmsh.model.geo.addPlaneSurface([ll3]) # fro
ll4 = gmsh.model.geo.addCurveLoop([c2, c12, -2, -c11])
s4 = gmsh.model.geo.addPlaneSurface([ll4]) # rig
ll5 = gmsh.model.geo.addCurveLoop([c3, c13, 3, -c12])
s5 = gmsh.model.geo.addPlaneSurface([ll5]) # bac 
ll6 = gmsh.model.geo.addCurveLoop([c4, c10, 4, -c13])
s6 = gmsh.model.geo.addPlaneSurface([ll6]) # lef


gmsh.model.geo.synchronize()  # must add synchronize before addPhysicalGroup
gmsh.model.addPhysicalGroup(2, [s1, s3, s4, s5, s6], 105)  # Absorbing boundary label
sl1 = gmsh.model.geo.addSurfaceLoop([s1, s3, s4, s5, s6, 1])
v1 = gmsh.model.geo.addVolume([sl1])

gmsh.model.geo.synchronize()
gmsh.model.addPhysicalGroup(3, [v1], 1)
gmsh.model.geo.synchronize()


# # Use the minimum of all the fields as the background mesh field
# gmsh.model.mesh.field.add("Min", 7)
# gmsh.model.mesh.field.setNumbers(7, "FieldsList", [2])

# gmsh.model.mesh.field.setAsBackgroundMesh(7)
gmsh.model.occ.synchronize()

gmsh.model.mesh.setSizeCallback(cal_mesh_size)

# gmsh.model.geo.synchronize()

gmsh.model.mesh.generate(3)

gmsh.write('model2_2hz_adapt_v1.msh2') # type 2 Gmsh file  

gmsh.finalize()

Info    : Increasing process stack size (8192 kB < 16 MB)
Info    : Creating geometry of discrete curves...
Info    : Done creating geometry of discrete curves (Wall 0.00181367s, CPU 0.000582s)
Info    : Creating geometry of discrete surfaces...
Info    : Done creating geometry of discrete surfaces (Wall 16.3625s, CPU 11.0875s)
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Discrete curve)
Info    : [ 10%] Meshing curve 2 (Discrete curve)
Info    : [ 20%] Meshing curve 3 (Discrete curve)
Info    : [ 30%] Meshing curve 4 (Discrete curve)
Info    : [ 40%] Meshing curve 5 (Line)
Info    : [ 50%] Meshing curve 6 (Line)
Info    : [ 50%] Meshing curve 7 (Line)
Info    : [ 60%] Meshing curve 8 (Line)
Info    : [ 70%] Meshing curve 9 (Line)
Info    : [ 80%] Meshing curve 10 (Line)
Info    : [ 90%] Meshing curve 11 (Line)
Info    : [100%] Meshing curve 12 (Line)
Info    : Done meshing 1D (Wall 2.29339s, CPU 1.94965s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Discret

In [221]:
# gmsh.finalize()
meshprefix = 'model2/model2_2hz_adapt_v1'

write_vtk(meshprefix)

In [198]:
# convert mesh into Paraview vtk file

import meshio

def write_vtk(meshprefix):
    mesh = meshio.read(
        meshprefix + '.msh2',  # string, os.PathLike, or a buffer/open file
        file_format="gmsh",  # optional if filename is a path; inferred from extension
    )
    
    meshio.write(
        meshprefix + ".vtk",  # str, os.PathLike, or buffer/ open file
        mesh,
        file_format="vtk",  # optional if first argument is a path; inferred from extension
    )

# mesh_in = pv.read('WelBasin.vtk')

# gmsh.finalize()

In [ ]:

## Adding fault plane
# lc2= 40 # 1e2
# p = gmsh.model.geo.addPoint(xmin+21500., ymin+17500., -3000., lc2)
# gmsh.model.geo.addPoint(xmin+23500., ymin+17500., -3000., lc2, p + 1)
# gmsh.model.geo.addPoint(xmin+23500., ymin+17500., -1000., lc2, p + 2)
# gmsh.model.geo.addPoint(xmin+21500., ymin+17500., -1000., lc2, p + 3)

# gmsh.model.occ.mesh.setSize([(0, p ), (0, p + 1), (0, p + 2), (0, p + 3)], lc2)

# l = gmsh.model.geo.addLine(p , p + 1)
# gmsh.model.geo.addLine(p + 1, p + 2, l + 1)
# gmsh.model.geo.addLine(p + 2, p + 3, l + 2)
# gmsh.model.geo.addLine(p + 3, p , l + 3)

# ll = gmsh.model.geo.addCurveLoop([l , l + 1, l + 2, l + 3])
# s = gmsh.model.geo.addPlaneSurface([ll])
# gmsh.model.addPhysicalGroup(2, [s], 103) # Fault plane label

# gmsh.model.geo.synchronize()
# gmsh.model.mesh.embed(2, [s], 3, 1) # Embed the fault plane to the Volume

## Element size
# lc_surf =0.1e3
# lc_max = 10e3
# gmsh.model.mesh.setSizeCallback(mesh_size)

# adaptive mesh size depending on distance

# gmsh.model.mesh.field.add("Distance", 1)
# # gmsh.model.mesh.field.setNumbers(1, "NodesList", [p1,p2,p3,p4]) # optional
# # gmsh.model.mesh.field.setNumber(1, "NNodesByEdge", 100)  # optional
# gmsh.model.mesh.field.setNumbers(1, "EdgesList", [1,2,3,4])
# gmsh.model.mesh.field.setNumbers(1, "FacesList", [1])

# gmsh.model.mesh.field.add("Threshold", 2)
# gmsh.model.mesh.field.setNumber(2, "IField", 1)
# gmsh.model.mesh.field.setNumber(2, "LcMin", lc_surf )#/ 20
# gmsh.model.mesh.field.setNumber(2, "LcMax", lc_max)
# gmsh.model.mesh.field.setNumber(2, "DistMin", 2000)
# gmsh.model.mesh.field.setNumber(2, "DistMax", 20000)

## Step 3. create Velocity model (NetCDF) file for SeisSol input
### Notes: original data from Matt Hill (GNS); series; interpolate to structured grids with designed spatial resolution (Nx, Ny, Nz).


In [169]:
# z_d[-1] = 10e3
# # 
# print(z_d)

<IPython.core.display.Javascript object>

### step 3.1. generate 3D velocity model


## 4. Generate Input kinematic source from SRF
### this is an example using SRF format source

In [10]:
# define the function from ~/SeisSol/SeisSol/preprocessing/science/kinematic_models/generate_FL33_input_files.py
# to read fault from srf and write a new one
# import Class FaultPlane based on module from https://github.com/SeisSol/Examples/blob/master/Northridge/generating_the_nrf.sh

from FaultPlaneWrite import FaultPlane

filename  = '/Users/duoli/Documents/SeisSol/Training/northridge/northridge_resampled.srf'
filename = '/Users/duoli/Documents/NSHM_SRM/Model_kinematic/northridge.srf'
outfile = 'northridge_format.srf'

p1 = FaultPlane()
p1.init_from_srf(filename)
p1.write_srf(outfile)


ImportError: cannot import name 'FaultPlane' from 'FaultPlaneWrite' (unknown location)

In [ ]:
## load source using SRF format

# print(p1.lon,p1.lat)

# origin of the source
lat_sou = -41.3
lon_sou = 174.75 

min_lat = -41.4
max_lat = -41.197
max_lon = 174.903 
min_lon = 174.7

# convert the source geometry to fit the domain area 
p1.lon = (p1.lon- p1.lon.min() )/(p1.lon.max()-p1.lon.min()) *  (max_lon-min_lon)/10 + lon_sou
p1.lat = (p1.lat -p1.lat.min()) /(p1.lat.max()-p1.lat.min()) *  (max_lat-min_lat)/10 + lat_sou
p1.depth = p1.depth-p1.depth + 2.0

print(p1.lon.shape,p1.lat.shape)

p1.write_srf(outfile)


In [238]:
# shfit to the current origin city of e.g. Wellingtone

# plot a figure showing the source 

fig, ax = plt.subplots(1,1,figsize=(7,5))
ax.imshow(topo, interpolation='bilinear', cmap='terrain',aspect='auto',
                   origin='lower', extent=[min_lon,max_lon,min_lat,max_lat],
                   vmax=700, vmin=-1000)

ax.contour(p1.lon,p1.lat,p1.slip1,cmap='viridis') # kinatic source

fig.colorbar(im2,ax=ax,label='Elevation (m)', fraction=0.046, pad=0.025, extend='both')
outname = 'TopoWel'+'-source.png'
plt.savefig(outname,dpi=150,transparent=False)
plt.show()

<IPython.core.display.Javascript object>

In [227]:
# print(p1.lon,p1.lat)

### 4.1 POint source

In [366]:
## plot source and receivers, as well as basin structure

# import seaborn as sbn
import matplotlib
matplotlib.style.use('seaborn-v0_8-talk') # Lovely plotting style

point1 = np.array([ 174.75, -41.3  ])
point1 = np.array([174.7223, -41.4383])
mypoint1 = pyproj.transform(lla,myproj, point1[0],point1[1])

# catalog
mypoint2 = pyproj.transform(lla,myproj,cat_subtable['Longitude'],cat_subtable['Latitude'])

# coastline
mypoint3= pyproj.transform(lla,myproj,coast[0,:],coast[1,:])

fig, ax = plt.subplots(1,1,figsize=(6,6))
ax.imshow(topo, interpolation='bilinear', cmap='gray',aspect='auto',
                   origin='lower', extent=[xrange[0],xrange[-1],yrange[0],yrange[-1]],
                   alpha=1.0,vmax=1500,vmin=-1500)

# data from load_vel_csv()
# im = ax.imshow(grdvel[75,:,:], interpolation='bilinear', cmap='magma',aspect='auto',
#                    origin='lower', extent=[grdx[0],grdx[-1],grdy[0],grdy[-1]],alpha=0.3,
#                    vmax=160, vmin= 700)

# ax.plot(mypoint1[0],mypoint1[1],'*',color='tomato')
cb1 =  ax.scatter(mypoint2[0],mypoint2[1],s=24,c=cat_subtable['Mw'],cmap='plasma_r')
ax.plot(mypoint3[0],mypoint3[1],'-',color='black',linewidth=1.2)

# ## basin edge stations
# # ax.plot(siteBasinEdge[::3,0],siteBasinEdge[::3,1],'^',color='royalblue',markersize=5.0)
# # stations
ax.scatter(x_site,y_site, s=23, marker='^', c='royalblue')

ax.set(xlim=(xrange[0]-10.e3,xrange[-1]+10.e3),ylim=(yrange[0]-10.e3,yrange[-1]+10.e3))

fig.colorbar(cb1,ax=ax,label='vs', fraction=0.046, pad=0.025, extend='both')

outname = 'TopoWel'+'-point2.png'

plt.savefig(outname, dpi=300,transparent=False)
plt.show()

/var/folders/w1/_6wbwxqn58j2p6t9v4n73wcr0000gn/T/ipykernel_57837/3079287416.py:9: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  mypoint1 = pyproj.transform(lla,myproj, point1[0],point1[1])
/var/folders/w1/_6wbwxqn58j2p6t9v4n73wcr0000gn/T/ipykernel_57837/3079287416.py:12: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  mypoint2 = pyproj.transform(lla,myproj,cat_subtable['Longitude'],cat_subtable['Latitude'])
/var/folders/w1/_6wbwxqn58j2p6t9v4n73wcr0000gn/T/ipykernel_57837/3079287416.py:15: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  mypoint3= pyproj.transform(lla,myproj,coast[0,:],coast[1,:])


<IPython.core.display.Javascript object>

In [364]:
# print（vs_3d[1]
# fig,ax=plt.subplots(1,1)
# ax.imshow(vs_3d[1,:,:],origin='lower', extent=[qx.min()*1e3+x1,qx.max()*1e3*x1,qy.min()*1e3+y1,qy.max()*1e3+y1])
# plt.savefig('test.png')
# plt.show()


<IPython.core.display.Javascript object>

In [170]:
# print(grdvel[75,:,:].min()) # check depth profile that show sh

coastfile ='../Geometry/CoastNorth.txt.npy'
coast =  np.load(coastfile)
print(coast.shape)

(2, 38487)


In [180]:
# load catalog data downloade from GeoNet

catfile = '/Users/duoli/Documents/NSHM_SRM/Model_kinematic/cat_subtable.csv'
# catfile = '/Users/duoli/Documents/NSHM_SRM/Model_kinematic/geonet_cat.csv'

cat_subtable = pd.read_csv(catfile)
print(cat_subtable.keys(),cat_subtable['Mw'])
# select earthquake in the research area
# exd = 0.1
# min_lat = -41.4-exd
# max_lat = -41.197 + exd

# max_lon = 174.903 +exd
# min_lon = 174.7-exd

# table1_select = table1[(table1['Longitude']>min_lon )& (table1['Longitude']<max_lon) & (table1['Latitude']>min_lat )& (table1['Latitude']<max_lat)]

# siteTable = pd.DataFrame(table1_select)
# siteTable.to_csv('cat_subtable.csv')
# print(table1_select.shape, print(siteTable.keys()))


Index(['Unnamed: 0', 'PublicID', 'Date', 'Latitude', 'Longitude', 'strike1',
       'dip1', 'rake1', 'strike2', 'dip2', 'rake2', 'ML', 'Mw', 'Mo', 'CD',
       'NS', 'DC', 'Mxx', 'Mxy', 'Mxz', 'Myy', 'Myz', 'Mzz', 'VR', 'Tva',
       'Tpl', 'Taz', 'Nva', 'Npl', 'Naz', 'Pva', 'Ppl', 'Paz', 'Method'],
      dtype='object') 0    4.7
1    4.3
2    3.8
3    4.0
4    4.3
5    3.9
6    4.0
7    3.5
8    3.4
Name: Mw, dtype: float64


## 5. Receivers around the Basin edge


In [ ]:
# how to prepare basin